# CIS Introduction

CIS has it's own version of the Iris Cube. But it's designed to work with any observational data. The CIS data structure is just called UngriddedData:

<img src="../images/ungridded_data.png" width="640"/>

First unzip your example data to a folder you can easily find

Be sure to activate your conda environment:

``$ source activate cis_env``

In [ ]:
# Ensure I don't use any local plugins. Set it to a readable folder with no Python files to avoid warnings.
%env CIS_PLUGIN_HOME=/Users/watson-parris/Pictures

In [ ]:
from cis import read_data, read_data_list, get_variables

get_variables('../resources/WorkshopData2016/ARCPAC_2008/*.nc')

In [ ]:
d = read_data("../resources/WorkshopData2016/ARCPAC_2008/*.nc", "NUMBER_CONCENTRATION")
print(d)

## Plotting

### Ungridded time series

In [ ]:
aeronet_aot_500 = read_data("../resources/WorkshopData2016/Aeronet/920801_150530_Brussels.lev20", 
                            "AOT_500")

In [ ]:
aeronet_aot_500.plot()

In [ ]:
ax = aeronet_aot_500.plot(color='red')
ax.set_yscale('log')

In [ ]:
aeronet_aot = read_data_list("../resources/WorkshopData2016/Aeronet/920801_150530_Brussels.lev20", 
                             ['AOT_500', 'AOT_675'])
ax = aeronet_aot.plot()

In [ ]:
ax.set_title('Brussels Aeronet AOT')
ax.set_xlabel('Date')

In [ ]:
from datetime import datetime
ax.set_xlim(datetime(2007,5,5), datetime(2007,8,26))

### Model time series

In [ ]:
model_aod = read_data("../resources/WorkshopData2016/od550aer.nc", "od550aer")
maod_global_mean, maod_std_dev, _ = model_aod.collapsed(['x', 'y'])

In [ ]:
ax = maod_global_mean.plot()

In [ ]:
aeronet_aot_500.plot(ax=ax)

In [ ]:
# This is a nice snippet which adds error bounds on the plot above

from cis.time_util import convert_std_time_to_datetime
ax.fill_between(convert_std_time_to_datetime(maod_global_mean.coord('time').points), maod_std_dev.data-maod_global_mean.data, maod_std_dev.data+maod_global_mean.data, alpha=0.3)

### Global model plot

In [ ]:
model_aod_annual_mean = model_aod.collapsed('t')

In [ ]:
model_aod_annual_mean[0].plot()

In [ ]:
model_aod_annual_mean[0].plot(central_longitude=180)

### Lat/lon aircraft plots

In [ ]:
number_concentration = read_data('../resources/WorkshopData2016/ARCPAC_2008', 
                                 'NUMBER_CONCENTRATION')
ax = number_concentration.plot()

In [ ]:
ax.bluemarble()

### Satellite plots

In [ ]:
aerosol_cci = read_data('../resources/WorkshopData2016/AerosolCCI', 
                        'AOD550')
aerosol_cci.plot()

In [ ]:
aerosol_cci_one_day = read_data('../resources/WorkshopData2016/AerosolCCI/20080415*.nc', 
                                'AOD550')
ax = aerosol_cci_one_day.plot()

In [ ]:
aerosol_cci_one_day.plot(projection='Orthographic')

In [ ]:
aerosol_cci_one_day.plot(projection='InterruptedGoodeHomolosine')

## Exercises

**1.** Try plotting ``AOT_500`` against ``AOT_675`` from the Aeronet file using a comparative scatter plot


**2.** Subset the 5 days of satellite data down to the region covered by the aircraft data, then plot it.

## Collocation

<img src="../images/collocation_options.png" width="640"/>

### Model onto Aeronet

<img src="../images/model_onto_aeronet.png" width="640"/>

This is an gridded onto un-gridded collocation and can be done using either linear interpolation or nearest neighbour.

This is very quick and in general CIS can even handle hybrid height coordinates: 

<img src="../images/gridded_ungridded_collocation.png" width="640"/>

In [ ]:
# Lets take a closer look at the model data
print(model_aod)

In [ ]:
from cis.time_util import PartialDateTime
# First subset the aeronet data:
aeronet_aot_2008 = aeronet_aot_500.subset(t=PartialDateTime(2008))

Note that we don’t actually have to do this subsetting, but that otherwise CIS will interpolate the nearest values, which in this case we don’t really want.

In [ ]:
# Now do the collocation:
model_aod_onto_aeronet = model_aod.collocated_onto(aeronet_aot_2008)

Note the updated history

In [ ]:
from cis.plotting.plot import multilayer_plot, taylor_plot
ax = multilayer_plot([model_aod_onto_aeronet[0], aeronet_aot_500_2008], 
                     layer_opts=[dict(label='Model'), 
                                 dict(label='Aeronet')], xaxis='time')

In [ ]:
taylor_plot([aeronet_aot_500_2008, model_aod_onto_aeronet[0]], 
            layer_opts=[dict(label='Aeronet'),dict(label='Model')])

In [ ]:
# Basic maths on the data
print(model_aod_onto_aeronet[0] - aeronet_aot_500_2008)

### Aircraft onto satellite

<img src="../images/aircraft_onto_satellite.png" width="640"/>

As you can see the difficulty here is the sparseness of the aircraft data, and actually of the satellite data in this region.

This is an ungridded to ungridded collocation:

<img src="../images/ungridded_ungridded_collocation.png" width="640" />

In [ ]:
# Read all of the AOD satelite variables
aerosol_cci = read_data_list('../resources/WorkshopData2016/AerosolCCI', 'AOD*0')
aoerosol_cci_Alaska = aerosol_cci.subset(x=[-170,-100],y=[35,80])

In [ ]:
aoerosol_cci_Alaska[0].plot(yaxis='latitude')

In [ ]:
aerosol_cci_collocated = aoerosol_cci_Alaska.collocated_onto(number_concentration, 
                                                             h_sep=10, t_sep='P1D')

In [ ]:
aerosol_cci_collocated.append(number_concentration)
print(aerosol_cci_collocated)

In [ ]:
aerosol_cci_collocated = aerosol_cci_collocated[::3]

In [ ]:
aerosol_cci_collocated[:2].plot('comparativescatter')

## Exercises

**1.** How does the correlation change if we only include those average number concentrations which averaged more than one point?

**2.** Consider the case of comparing our model AOD with the AerosolCCI.

**a.** What strategies could you employ?
    
**b.** Perform an initial assesment of the model AOD field using the Aerosol CCI data for the few days we have data.

## CIS and Pandas

In [ ]:
df = aerosol_cci_collocated.as_data_frame()
print(df)

In [ ]:
df.corr()
# Then do a pretty plot of it...
# This is a nice segway into the Pandas lesson.

In [ ]:
# Save the collocation output so that we can come back to it during the Pandas tutorial.
aerosol_cci_collocated.save_data('col_output.nc')